<a href="https://colab.research.google.com/github/newtonxp/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ChatBot**

- Meet my Python chatbot, trained on neural networks, designed to give you a brief insight into who I am.
- This interactive bot engages in conversation, sharing essential details about my educational and professional background, expertise, and interests.
- With its natural language processing capabilities, it ensures a seamless experience while responding to various queries. Ask about my education, work, or hobbies, and let this chatbot introduce me in a unique and engaging way.

**NOTE:** Due to some bugs in tflearn, it does not work well with the recent python update. So, you'll need to install python 3.6 for this project.

In [1]:
# first install python 3.6
!sudo apt-get update -y
!sudo apt-get install python3.6
# change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
# select python version
!sudo update-alternatives --config python3
# check python version
!python --version
# install pip for new python
!sudo apt-get install python3.6-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
# upgrade pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [43.3 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [850 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.c

In [2]:
!pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127282 sha256=4497014674ca058041cb56fc545f0818dd89e98743693b67460841b6e40848f5
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import pandas as pd
import tflearn
import tensorflow
import json
import random
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
stemmer = LancasterStemmer()

data = pd.read_json('intents.json')


In [11]:
# getting the questions and answers from the json file

try:
  with open("data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

except:
  words = []
  labels = []
  docs_x = []
  docs_y =[]

  for intent in data["intents"]:
      for pattern in intent["patterns"]:
          wrds = nltk.word_tokenize(pattern)
          words.extend(wrds)
          docs_x.append(wrds)
          docs_y.append(intent["tag"])

      if intent["tag"] not in labels:
          labels.append(intent["tag"])

  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
      bag = []
      wrds = [stemmer.stem(w) for w in doc]

      for w in words:
          if w in wrds:
              bag.append(1)
          else:
              bag.append(0)
      output_row = out_empty[:]
      output_row[labels.index(docs_y[x])] = 1

      training.append(bag)
      output.append(output_row)

  training = np.array(training)
  output = np.array(output)

  with open("data.pickle", "wb") as f:
      pickle.dump((words, labels, training, output), f)


# **Training the model**

- For training, we will use the Deep Neural Network from tflearn module.
- We have a total of 4 layers where the first layer is the input layer, the last one id the output layer and the moddle ones are the hidden layers each with 8 neurons.
- In each layer, all neurons are interconnected, forming a 'Fully Connected' architecture where every neuron is linked to all neurons in the subsequent layer.

In [6]:
# training the model using different layers

tensorflow.compat.v1.reset_default_graph()


net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=500, batch_size=9, show_metric=True)

Training Step: 2499  | total loss: 0.04509 | time: 0.025s
| Adam | epoch: 500 | loss: 0.04509 - acc: 1.0000 -- iter: 36/38
Training Step: 2500  | total loss: 0.04567 | time: 0.032s
| Adam | epoch: 500 | loss: 0.04567 - acc: 1.0000 -- iter: 38/38
--


In [7]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  return np.array(bag)

In [10]:
def chat():
  print("Start chatting with the bot or QUIT to stop")

  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      print("Thank you for your time. Have a nice dasy!")
      break

    results = model.predict([bag_of_words(inp, words)])[0]
    results_index = np.argmax(results)
    tag = labels[results_index]

    if results[results_index] > 0.7:
      for tg in data["intents"]:
        if tg['tag'] == tag:
          responses = tg['responses']
      print(random.choice(responses))

    else:
      print("I didn't understand. Please ask another question.")




chat()

Start chatting with the bot or QUIT to stop
You: hi
Nice to meet you!
You: what is your name
My name is Jay.
You: what is your age
23 years young!
You: List some of the programming languages you are familiar with
In the field of Data Science and Machine Learning, I possess expertise in several programming languages. Among them, I excel in Python, C, C++, Java, R, MATLAB, MySQL, and MySQLite. 
You: What is your proficiency in different AI/ML tools?
Through internships and relevant coursework focused on Data Science and Machine Learning, I am proficient in Microsoft Suite, Numpy, Pandas, Matplotlib, Scikit-learn, Tensorflow, PyTorch, Keras, Spacy, NLP, and Tableau.
You: Tell me about your work experience
I've worked at Free Co Educations as a Data Scientist Intern and at KYC Hub as a Machine Learning Intern
You: quit
Thank you for your time. Have a nice dasy!


# **Conclusions**
- **Interactive Introduction:** Explore the world of conversational AI with our Python chatbot. This repository showcases a chatbot designed to introduce myself in an engaging and interactive manner. Through natural language processing, users can initiate conversations and learn about my background, education, work experiences, and more.

- **Neural Network Powered:** Built on the foundation of neural networks, this chatbot demonstrates the potential of AI-driven conversations. Its neural network architecture enables smooth interactions, providing personalized responses to a variety of questions. By combining neural networks with natural language processing, the chatbot offers a seamless user experience.

- **Customizable and Educational:** Feel free to customize and extend this chatbot for your own purposes. Whether you're a developer looking to learn about chatbot development or an enthusiast curious about the capabilities of AI, this repository serves as a valuable resource. Dive into the code, experiment with different responses, and gain insights into creating engaging chatbot interactions.

# **Further Reading**
- If you want to tweak some parameters in the deep neural network, you can check the link http://tflearn.org/models/dnn/
- Here, you will find additional details regarding the hyperparameters that can be adjusted to enhance performance.